# OpenAI Function Calling

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import json

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }
    
    return json.dumps(weather_info)

### Defining a function

In [3]:
functions = [
    {
        "name": "get_current_weather",
        "description": """Get the current weather in a given location""",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "description": "The unit of temperature",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

### Calling OpenAI API with function definition

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in New Delhi?"
    }
]

In [5]:
import openai

In [6]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [7]:
print(response)

{
  "id": "chatcmpl-8QrcqfPWR2Qkmc1RQ6Oed7iAt2ab4",
  "object": "chat.completion",
  "created": 1701413356,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"New Delhi\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 17,
    "total_tokens": 106
  },
  "system_fingerprint": null
}


In [8]:
response_message = response["choices"][0]["message"]
response_message

<OpenAIObject at 0x7fc3fc2c4220> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"New Delhi\"\n}"
  }
}

### Calling function with returned kwargs from OpenAI API

In [10]:
kwargs = json.loads(response_message["function_call"]["arguments"])
kwargs

{'location': 'New Delhi'}

In [11]:
get_current_weather(**kwargs)

'{"location": "New Delhi", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

### Case when no function is required

In [15]:
messages = [
    {"role": "user", "content": "hi!"}
]

In [16]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)
response

<OpenAIObject chat.completion id=chatcmpl-8QrnnyLVmnuOEPtQmiU70uENB8JaY at 0x7fc3fc2bff40> JSON: {
  "id": "chatcmpl-8QrnnyLVmnuOEPtQmiU70uENB8JaY",
  "object": "chat.completion",
  "created": 1701414035,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 10,
    "total_tokens": 92
  },
  "system_fingerprint": null
}

### Three modes of function calling

- `auto`
- `none`
- `{"name": "<function_name>"}`

In [19]:
messages = [
    {"role": "user", "content": "What's the weather like in New Delhi?"}
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none"
)
response

<OpenAIObject chat.completion id=chatcmpl-8QroRNynkcmCezJmo6C324hDqQK5T at 0x7fc3fc2e67c0> JSON: {
  "id": "chatcmpl-8QroRNynkcmCezJmo6C324hDqQK5T",
  "object": "chat.completion",
  "created": 1701414075,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Let me check the current weather in New Delhi for you."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 90,
    "completion_tokens": 12,
    "total_tokens": 102
  },
  "system_fingerprint": null
}

In [20]:
messages = [
    {"role": "user", "content": "What's the weather like in New Delhi?"}
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto"
)
response

<OpenAIObject chat.completion id=chatcmpl-8Qrop4LxYoAb70lGmjI8vOiYG2Epe at 0x7fc3fc2c4d10> JSON: {
  "id": "chatcmpl-8Qrop4LxYoAb70lGmjI8vOiYG2Epe",
  "object": "chat.completion",
  "created": 1701414099,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"New Delhi\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 17,
    "total_tokens": 106
  },
  "system_fingerprint": null
}

In [21]:
messages = [
    {"role": "user", "content": "Hello, how are you?"}
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
response

<OpenAIObject chat.completion id=chatcmpl-8QrpHtq22MHoJdX0E7v1GcK3xWnY9 at 0x7fc3fc2e9540> JSON: {
  "id": "chatcmpl-8QrpHtq22MHoJdX0E7v1GcK3xWnY9",
  "object": "chat.completion",
  "created": 1701414127,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 93,
    "completion_tokens": 12,
    "total_tokens": 105
  },
  "system_fingerprint": null
}

### Passing back function value to OpenAI API

In [26]:
messages = [
    {"role": "user", "content": "What's the weather like in New Delhi?"}
]

In [27]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather of the location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Name of the city to fetch the weather for. E.g. New Delhi"
                },
                "unit": {
                    "type": "string",
                    "description": "Unit of temperature",
                    "enum": ["fahrenheit", "celcius"]
                }
            }
        }
    }
]

In [28]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [29]:
response

<OpenAIObject chat.completion id=chatcmpl-8Qs2HnbCaxOPGQX4RP95gGgY8ZHES at 0x7fc3fc2c4e50> JSON: {
  "id": "chatcmpl-8Qs2HnbCaxOPGQX4RP95gGgY8ZHES",
  "object": "chat.completion",
  "created": 1701414933,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"New Delhi\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 91,
    "completion_tokens": 17,
    "total_tokens": 108
  },
  "system_fingerprint": null
}

In [35]:
kwargs = json.loads(response["choices"][0]["message"]["function_call"]["arguments"])
kwargs

{'location': 'New Delhi'}

In [36]:
observation = get_current_weather(**kwargs)
observation

'{"location": "New Delhi", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [37]:
messages.append(
{
    "role": "function",
    "name": "get_current_weather",
    "content": observation
})

In [38]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages
)
print(response)

{
  "id": "chatcmpl-8Qs5ixKb7ELV3mLLYtNk38vVJTqsE",
  "object": "chat.completion",
  "created": 1701415146,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in New Delhi is currently sunny and windy with a temperature of 72\u00b0F (22\u00b0C)."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 53,
    "completion_tokens": 21,
    "total_tokens": 74
  },
  "system_fingerprint": null
}
